# Indirect Zero-Noise-Extrapolation
---

This is our configuration:

```yml
nqubit: 5
layer: 10
connstraint: Flase
```

### Linear regression

The general single variable linear regression: $E_{linear}(\lambda)=c_0 + c_1\lambda$

Multivariate linear regression:

$$E_{linear}(n_R, n_T, nY)=c_0 + c_1 n_R + c_2 n_T + c_3 n_Y$$

Where independent variables are: $n_R, n_T, n_Y$

In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import itertools
import os
import pickle
import random
from collections import Counter, defaultdict
from copy import deepcopy
from typing import Any, Callable, Generator, Optional
# Functional programming utilities.
import functools

from sympy import *
init_printing(use_unicode=True)

In [14]:
def get_monomials(n: int, d: int) -> list[str]:
    """Compute monomials of degree `d` in graded lexicographical order."""
    variables = [f"λ_{i}" for i in range(1, n + 1)]
    
    monomials = []
    for degree in range(d, -1, -1):
        # Generate combinations for the current degree
        combos = list(itertools.combinations_with_replacement(variables, degree))
        
        # Sort combinations lexicographically
        combos.sort()
        
        # Construct monomials from sorted combinations
        for combo in combos:
            monomial_parts = []
            counts = Counter(combo)
            # Ensure variables are processed in lexicographical order
            for var in sorted(counts.keys()):
                count = counts[var]
                if count > 1:
                    monomial_parts.append(f"{var}**{count}")
                else:
                    monomial_parts.append(var)
            monomial = "*".join(monomial_parts)
            # Handle the case where degree is 0
            if not monomial:
                monomial = "1"
            monomials.append(monomial)
    # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
    return monomials[::-1]

def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:
    """Construct a matrix from monomials evaluated at sample points."""
    n = len(sample_points[0])  # Number of variables based on the first sample point
    monomials = get_monomials(n, degree)
    matrix = np.zeros((len(sample_points), len(monomials)))

    for i, point in enumerate(sample_points):
        for j, monomial in enumerate(monomials):
            var_mapping = {f"λ_{k+1}": point[k] for k in range(n)}
            matrix[i, j] = eval(monomial, {}, var_mapping)
    return matrix

def get_eta_coeffs_from_sample_matrix(mat: np.ndarray) -> list[float]:
    """Given a sample matrix compute the eta coefficients."""
    n_rows, n_cols = mat.shape
    if n_rows != n_cols:
        raise ValueError("The matrix must be square.")

    det_m = np.linalg.det(mat)    
    if det_m == 0:
        raise ValueError("The matrix is singular.")
    
    terms = []
    for i in range(n_rows):
        new_mat = mat.copy()
        new_mat[i] = np.array([[0] * (n_cols - 1) + [1]])        
        terms.append(np.linalg.det(new_mat) / det_m)

    return terms

def get_eta_coeffs(lre_scale_factors: list[float], degree: int) -> list[float]:
    """Obtain eta coefficients from list of scale factors and degree."""
    mat = sample_matrix(sample_points=lre_scale_factors, degree=degree)
    return get_eta_coeffs_from_sample_matrix(mat)

def get_eta_coeffs_single_variable(scale_factors: list[float]) -> list[float]:
    """Returns the array of single-variable Richardson extrapolation coefficients associated
    to the input array of scale factors."""
    # Lagrange interpolation formula.
    richardson_coeffs = []
    for l in scale_factors:
        coeff = 1.0
        for l_prime in scale_factors:
            if l_prime == l:
                continue
            coeff *= l_prime / (l_prime - l)
        richardson_coeffs.append(coeff)

    return richardson_coeffs

In [26]:
# sample_points = [ (4, 1, 0), (12, 3, 1), (20, 5, 2), (28, 7, 3), (36, 9, 4), (44, 11, 5), (52, 13, 6), (60, 15, 7), (68, 17, 8), (76, 19, 9), (84, 21, 10), (92, 23, 11), (100, 25, 12), (108, 27, 13), (116, 29, 14), (124, 31, 15), (132, 33, 16), (140, 35, 17), (148, 37, 18), (156, 39, 19)]
# sample_points = [(4, 1, -5.907748909320789, 0),
# (12, 1, -5.916920204491979, 0),
# (12, 3, -5.800746641641066, 1),
# (20, 3, -5.764809805003449, 1),
# (20, 5, -5.679828979633399, 2)
# ] 

In [59]:
n, d = 2, 1
sample_points = [
    (4, 1), (12, 5), (20, 5)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")

Monomials: ['1', 'λ_2', 'λ_1']
Monomials: 3


In [60]:
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

⎡1.0  1.0  4.0 ⎤
⎢              ⎥
⎢1.0  5.0  12.0⎥
⎢              ⎥
⎣1.0  5.0  20.0⎦

In [73]:
m = np.linalg.det(mat)

In [74]:
m

In [75]:
from sympy import Matrix

# Define the matrix
m1 = Matrix([
    [1, Symbol('λ1'), Symbol('λ2')],
    [1, 12, 5],
    [1, 20, 5]
])

# Calculate determinant
det1 = m1.det()

print(det1/m)


0.25*λ2 - 1.25


In [76]:
# Define the matrix
m2 = Matrix([
    [1, 4, 1],
    [1, Symbol('λ1'), Symbol('λ2')],
    [1, 20, 5]
])

# Calculate determinant
det2 = m2.det()

print(det2/m)

0.125*λ1 - 0.5*λ2


In [77]:
m3 = Matrix([
    [1, 4, 1],
    [1, 12, 5],
    [1, Symbol('λ1'), Symbol('λ2')]
])

# Calculate determinant
det3 = m3.det()

print(det3/m)

-0.125*λ1 + 0.25*λ2 + 0.25


In [79]:
print((det1/32)+(det2/32)+(det3/32))

-1


In [84]:
n, d = 2, 2
sample_points = [
    (4, 1), (12, 7), (36, 5), (36, 13), (52, 5)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")


Monomials: ['1', 'λ_2', 'λ_1', 'λ_2**2', 'λ_1*λ_2', 'λ_1**2']
Monomials: 6


In [85]:
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

⎡1.0  1.0   4.0    1.0    4.0    16.0 ⎤
⎢                                     ⎥
⎢1.0  5.0   12.0  25.0   60.0   144.0 ⎥
⎢                                     ⎥
⎢1.0  9.0   20.0  81.0   180.0  400.0 ⎥
⎢                                     ⎥
⎢1.0  7.0   28.0  49.0   196.0  784.0 ⎥
⎢                                     ⎥
⎢1.0  9.0   36.0  81.0   324.0  1296.0⎥
⎢                                     ⎥
⎣1.0  11.0  44.0  121.0  484.0  1936.0⎦

In [86]:
np.linalg.det(mat)